<a href="https://colab.research.google.com/github/Enes-Shaltami/Anagram_Solver/blob/master/Message.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import simpy

In [ ]:
!pip install simpy

In [ ]:
def car(env):
  while True:
    print('Start parking at %d' % env.now)
    parking_duration = 5
    yield env.timeout(parking_duration)
    print('Start driving at %d' % env.now)
    trip_duration = 2
    yield env.timeout(trip_duration)

env = simpy.Environment()
env.process(car(env))
env.run(until=15)



Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [ ]:
#### ITERATION #1
class Process:
  domain = 0
  process_id = -1
  state = "READY"

  def __init__(self, process_id, domain):
    self.process_id = process_id
    self.domain = domain

  def setState(state):
    self.state = state


NEW_PROC = 3
BLOCK_PROC = 5

blocked_queue = []
time_queue = []
work_queue = []
work_queue.append(Process(1, "ARM"))
work_queue.append(Process(2, "x86"))
work_queue.append(Process(3, "RISC-V"))
work_queue.append(Process(4, "MIPS"))


def Process_run(env, name, bcs, new_proc, interrupt):
  # Simulate driving to the BCS
  yield env.timeout(new_proc)

  # Make a request
  print('%s ready at %d' % (name, env.now))
  with bcs.request() as req:
    yield req # return the process with the curretn request

  # Run the program
    print('%s running its program at %s' % (name, env.now))
    yield env.timeout(interrupt) # insert delay for interrupt
    print('%s blocked from program at %s' % (name, env.now))


env = simpy.Environment() # execution environment for event-based simulation
bcs = simpy.Resource(env, capacity=1) # resources for environment with a capacity of processes

for i in range(6): # instantiate 10 processes
  env.process(Process_run(env, 'Process %d' % i, bcs, i*NEW_PROC, BLOCK_PROC))
  
# env.process(env, work_queue.pop(0), bcs, )

env.run()


Process 0 ready at 0
Process 0 running its program at 0
Process 1 ready at 3
Process 0 blocked from program at 5
Process 1 running its program at 5
Process 2 ready at 6
Process 3 ready at 9
Process 1 blocked from program at 10
Process 2 running its program at 10
Process 4 ready at 12
Process 5 ready at 15
Process 2 blocked from program at 15
Process 3 running its program at 15
Process 3 blocked from program at 20
Process 4 running its program at 20
Process 4 blocked from program at 25
Process 5 running its program at 25
Process 5 blocked from program at 30


In [ ]:
#### ITERATION #2
import random
import math
class Process:
  domain = 0
  process_id = -1
  state = "READY"
  readyTime = 0
  runningTime = 0
  blockedTime = 0

  def __init__(self, process_id, domain):
    self.process_id = process_id
    self.domain = domain

  def setState(self, state):
    self.state = state

  def setRunningTime(self, time):
    self.runningTime = time

  def setReadyTime(self, time):
    self.readyTime = time

  def setBlockedTime(self, time):
    self.blockedTime = time
  
  def getReadyTime(self):
    return self.readyTime
  def getProcessID(self):
    return self.process_id

def exponential_probability():
  lam = 1
  x = random.randrange(0, 10)
  return lam * math.exp(-1 * lam * x) * 1000


NEW_PROC = 3
BLOCK_PROC = 5

blocked_queue = []
time_queue = []
work_queue = []
work_queue.append(Process(1, "ARM"))
work_queue.append(Process(2, "x86"))
work_queue.append(Process(3, "RISC-V"))
work_queue.append(Process(4, "MIPS"))


def Process_run(env, process, bcs):
  # Simulate driving to the BCS
  # yield env.timeout(5)
  yield env.timeout(process.getReadyTime())

  # # Make a request
  print('Process ID %s ready at %d' % (process.getProcessID(), env.now))
  with bcs.request() as req:
    yield req # return the process with the curretn request

  # # Run the program
    print('Process ID %s running its program at %s' % (process.getProcessID(), env.now))
    blocked_queue.append(process)
    yield env.timeout(math.floor(exponential_probability())) # insert delay for interrupt
    print('Process ID %s blocked from program at %s' % (process.getProcessID(), env.now))

env = simpy.Environment() # execution environment for event-based simulation
bcs = simpy.Resource(env, capacity=1) # resources for environment with a capacity of processes

# for i in range(6): # instantiate 10 processes
#   env.process(Process_run(env, 'Process %d' % i, bcs, i*NEW_PROC, BLOCK_PROC))
  
current_work = work_queue.pop(0)
env.process(Process_run(env, current_work, bcs))

current_work = work_queue.pop(0)
env.process(Process_run(env, current_work, bcs))
current_work = work_queue.pop(0)
env.process(Process_run(env, current_work, bcs))

env.run()
print(exponential_probability())

# current_work = work_queue.pop(0)
# current_work.getReadyTime()


NameError: ignored

In [ ]:
from functools import partial, wraps
import simpy

def trace(env, callback):
  """Replace the ``step()`` method of *env* with a tracing function
  that calls *callbacks* with an events time, priority, ID and its
  instance just before it is processed.

     """
  def get_wrapper(env_step, callback):
    """Generate the wrapper for env.step()."""
    @wraps(env_step)
    def tracing_step():
    """Call *callback* for the next event if one exist before
    calling ``env.step()``."""
      if len(env._queue):
        t, prio, eid, event = env._queue[0]
        callback(t, prio, eid, event)
      return env_step()
    return tracing_step

  env.step = get_wrapper(env.step, callback)

def monitor(data, t, prio, eid, event):
  data.append((t, eid, type(event)))

def test_process(env):
  yield env.timeout(1)

data = []
# Bind *data* as first argument to monitor()
# see https://docs.python.org/3/library/functools.html#functools.partial
monitor = partial(monitor, data)

env = simpy.Environment()
trace(env, monitor)

p = env.process(test_process(env))
env.run(until=p)

for d in data:
  print(d)


IndentationError: ignored